Solving a Finite Markov Decision Processes with Q-Learning:
game with map like environment, treasure, trees, monsters, obstacles, rewards

https://medium.com/@curiousily/solving-an-mdp-with-q-learning-from-scratch-deep-reinforcement-learning-for-hackers-part-1-45d1d360c120

https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

In [1]:
import numpy as np
import random

Building the environment

In [2]:
MONSTER = "m"
PLAYER = "p"
REWARD = "r"
EMPTY = "*"
TREE = "t"

grid = [
    [REWARD, EMPTY],
    [MONSTER, PLAYER]
]

for row in grid:
    print(' '.join(row))

r *
m p


Wrap environment state in a class

In [3]:
class State:
    
    def __init__(self, grid, player_pos):
        self.grid = grid
        self.player_pos = player_pos
        
    def __eq__(self, other):
        return isinstance(other, State) and self.grid == other.grid and self.player_pos == other.player_pos
    
    def __hash__(self):
        return hash(str(self.grid) + str(self.player_pos))
    
    def __str__(self):
        return f"State(grid={self.grid}, player_pos={self.player_pos})"

Actions and init state

In [4]:
UP = 0
DOWN = 1
LEFT = 2
RIGHT = 3

ACTIONS = [UP, DOWN, LEFT, RIGHT]

start_state = State(grid=grid, player_pos=[1, 1])

Function that takes the current state with an action and returns new state, reward and whether or not the episode has completed


In [5]:
from copy import deepcopy

def act(state, action):
    
    def new_player_pos(state, action):
        p = deepcopy(state.player_pos)
        if action == UP:
            p[0] = max(0, p[0] - 1)
        elif action == DOWN:
            p[0] = min(len(state.grid) - 1, p[0] + 1)
        elif action == LEFT:
            p[1] = max(0, p[1] - 1)
        elif action == RIGHT:
            p[1] = min(len(state.grid[0]) - 1, p[1] + 1)
        else:
            raise ValueError(f"Unknown action {action}")
        return p
            
    p = new_player_pos(state, action)
    grid_item = state.grid[p[0]][p[1]]
    
    new_grid = deepcopy(state.grid)
    
    if grid_item == MONSTER:
        reward = -100
        is_done = True
        new_grid[p[0]][p[1]] += PLAYER
    elif grid_item == REWARD:
        reward = 1000
        is_done = True
        new_grid[p[0]][p[1]] += PLAYER
    elif grid_item == EMPTY:
        reward = -1
        is_done = False
        old = state.player_pos
        new_grid[old[0]][old[1]] = EMPTY
        new_grid[p[0]][p[1]] = PLAYER
    elif grid_item == PLAYER:
        reward = -1
        is_done = False
    else:
        raise ValueError(f"Unknown grid item {grid_item}")
    
    return State(grid=new_grid, player_pos=p), reward, is_done

Learning

In [6]:
import numpy as np
import random

random.seed(42) 

N_STATES = 4
N_EPISODES = 20

MAX_EPISODE_STEPS = 100

MIN_ALPHA = 0.02

alphas = np.linspace(1.0, MIN_ALPHA, N_EPISODES)
gamma = 1.0
eps = 0.2

q_table = dict()

A helper function q that gives the Q value for a state-action pair or for all actions, given a state

In [7]:
def q(state, action=None):
    
    if state not in q_table:
        q_table[state] = np.zeros(len(ACTIONS))
        
    if action is None:
        return q_table[state]
    
    return q_table[state][action]

Act with random action with some small probability or the best action seen so far (using our q_table)

In [8]:
def choose_action(state):
    if random.uniform(0, 1) < eps:
        return random.choice(ACTIONS) 
    else:
        return np.argmax(q(state))

Training agent using the Q-learning algorithm

In [9]:
for e in range(N_EPISODES):
    
    state = start_state
    total_reward = 0
    alpha = alphas[e]
    
    for _ in range(MAX_EPISODE_STEPS):
        action = choose_action(state)
        next_state, reward, done = act(state, action)
        total_reward += reward
        
        q(state)[action] = q(state, action) + \
                alpha * (reward + gamma *  np.max(q(next_state)) - q(state, action))
        state = next_state
        if done:
            break
    print(f"Episode {e + 1}: total reward -> {total_reward}")

Episode 1: total reward -> 999
Episode 2: total reward -> 998
Episode 3: total reward -> 997
Episode 4: total reward -> 997
Episode 5: total reward -> 999
Episode 6: total reward -> 999
Episode 7: total reward -> 998
Episode 8: total reward -> -100
Episode 9: total reward -> -101
Episode 10: total reward -> 999
Episode 11: total reward -> 999
Episode 12: total reward -> 999
Episode 13: total reward -> 999
Episode 14: total reward -> 999
Episode 15: total reward -> 999
Episode 16: total reward -> 998
Episode 17: total reward -> 999
Episode 18: total reward -> 999
Episode 19: total reward -> 999
Episode 20: total reward -> 999


In [10]:
r = q(start_state)
print(f"up={r[UP]}, down={r[DOWN]}, left={r[LEFT]}, right={r[RIGHT]}")

up=998.9999564956789, down=225.12936017286486, left=-85.10182825484765, right=586.1924520375833


In [11]:
new_state, reward, done = act(start_state, UP)

In [12]:
r = q(new_state)
print(f"up={r[UP]}, down={r[DOWN]}, left={r[LEFT]}, right={r[RIGHT]}")

up=895.9452631578947, down=842.8767095010014, left=1000.0, right=967.1072709141274
